# Line indicators

In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import talib as ta
from lightweight_charts import Chart
import asyncio
import nest_asyncio
from websockets import Close

nest_asyncio.apply()

In [2]:
df = yf.download('TQQQ', start='2020-01-01', multi_level_index=False)
df.reset_index(inplace=True)

df.head()

C:\Users\jwang\AppData\Local\Temp\ipykernel_22140\4271637344.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download('TQQQ', start='2020-01-01', multi_level_index=False)
[*********************100%***********************]  1 of 1 completed


,Date,Close,High,Low,Open,Volume
0,2020-01-02,21.760862,21.760862,21.133238,21.250618,65536000
1,2020-01-03,21.169168,21.559637,20.898474,20.910452,72590000
2,2020-01-06,21.571615,21.578801,20.589453,20.666109,64047600
3,2020-01-07,21.545271,21.746493,21.358421,21.593181,53849600
4,2020-01-08,22.041138,22.345367,21.447049,21.550056,79582400


In [3]:
if __name__ =='__main__':
    chart = Chart(title='TQQQ with EMA 12 and EMA 25', maximize=True)
    chart.legend(visible=True)

    chart.set(df)

    ema12_line = chart.create_line('EMA 12', color = '#ffeb3b', width = 1, price_label = True)
    ema12 = pd.DataFrame(columns = ['time', 'EMA 12'])
    ema12['time'] = df.Date
    ema12['EMA 12'] = ta.EMA(df['Close'], 12)
    ema12_line.set(ema12.dropna())

    ema25_line = chart.create_line('EMA 25', color = '#26c6da', width = 1, price_label = True)
    ema25 = pd.DataFrame(columns = ['time', 'EMA 25'])
    ema25['time'] = df.Date
    ema25['EMA 25'] = ta.EMA(df['Close'], 25)
    ema25_line.set(ema25.dropna())

    chart.show(block=True)


In [4]:
# Draw Bollinger Bands
chart = Chart(title='TQQQ with Bollinger Bands', maximize=True)
chart.legend(visible=True)

chart.set(df)

# Calculate Bollinger Bands
# Default timeperiod=20, nbdevup=2, nbdevdn=2
upper_band, middle_band, lower_band = ta.BBANDS(df['Close'], timeperiod=20, nbdevup=2, nbdevdn=2)

# Add the bands to your dfFrame
df['upper_band'] = upper_band
df['middle_band'] = middle_band
df['lower_band'] = lower_band

# Drop rows with NaN values resulting from the calculation
df.dropna(inplace=True)


upper_line = chart.create_line('UpperBB', color = 'red', width = 1, price_label = True)
upper = pd.DataFrame(columns = ['time', 'UpperBB'])
upper['time'] = df.Date
upper['UpperBB'] = upper_band
upper_line.set(upper.dropna())

middle_line = chart.create_line('MiddleBB', color = 'orange', width = 1, price_label = True)
middle = pd.DataFrame(columns = ['time', 'MiddleBB'])
middle['time'] = df.Date
middle['MiddleBB'] = middle_band
middle_line.set(middle.dropna())

lower_line = chart.create_line('LowerBB', color = 'green', width = 1, price_label = True)
lower = pd.DataFrame(columns = ['time', 'LowerBB'])
lower['time'] = df.Date
lower['LowerBB'] = lower_band
lower_line.set(lower.dropna())

# Show the chart
chart.show(block=True)

In [5]:

# Calculate the RSI. TA-Lib expects NumPy arrays.
df['rsi'] = ta.RSI(df['Close'], timeperiod=14)

# Drop rows with NaN values resulting from the RSI calculation
df.dropna(subset=['rsi'], inplace=True)

In [10]:
if __name__ == '__main__':
    # Create the main candlestick chart
    chart = Chart(title='TQQQ with RSI Indicator', maximize=True, inner_height=0.7)
    chart.time_scale(visible=False)
    chart.set(df)

    # Create a sub-chart for the RSI indicator
    rsi_chart = chart.create_subchart(height=0.3, width=1, sync=True)
    rsi_line = rsi_chart.create_line('rsi', color = 'red', width = 1, price_label = True)
    rsi_line.set(df[['Date', 'rsi']].dropna())


    chart.show(block=True)

In [11]:
rsi.head()

,time,RSI14
33,2020-02-20,72.341623
34,2020-02-21,60.556285
35,2020-02-24,45.420530
36,2020-02-25,38.913888
37,2020-02-26,40.352446
